In [ ]:
# Code not fully functional in this state
# Only for display/appendix

In [ ]:
from scipy.integrate import solve_ivp
import pysindy as ps
import matplotlib.pyplot as plt
from scipy.io import loadmat
import random
from scipy.optimize import minimize

In [ ]:
# Gaussian noise addition
import numpy as np
from sklearn.metrics import mean_squared_error
def add_noise(data, percentage=0):
    rmse = mean_squared_error(u, np.zeros(data.shape), squared=False)
    data = data + np.random.normal(0, (percentage/100)*rmse, u.shape)
    return data

In [ ]:
# Savitzky-Golay Filter 
from scipy.signal import savgol_filter
def savgol_denoise_1D(data, window_length=5,poly_order=0):
    denoised_data = savgol_filter(data, window_length, poly_order)  
    return denoised_data

In [ ]:
# l1 Trend Filter
import cvxpy as cp
def l1_trend_filter(y, lamb=0):
    n = len(y)
    x = cp.Variable(n)
    # Construct 2nd order difference matrix D2
    D = np.diff(np.eye(n), axis=0)
    D2 = np.diff(D, axis=0)
    # Define the objective
    objective = cp.Minimize(0.5 * cp.norm(x - y, 2)**2 + 
                            lamb * cp.norm(D2 @ x, 1))
    # Construct the problem
    problem = cp.Problem(objective)
    # Apply ECOS solver
    try:
        problem.solve()
    except cp.SolverError:
        # Apply SCS solver if ECOS fails
        try:
            problem.solve(solver=cp.SCS)
        # No solution found
        except cp.SolverError:
            return None
    # Return the solution 
    return x.value

In [ ]:
# Pareto selection of Savitzky-Golay window size
# Lorenz window range: 5 to 229, degree: 2
# Burgers window range: 5 to 99, degree: 3
def pareto_savgol_w(u_single):
    # Initialization
    windows = [i for i in range(5, 229, 2)]
    solution_norms = []
    residual_norms = []
    
    # Calculate norms for each window after filter application
    for w in windows:
        u_denoised = savgol_denoise_1D(u_single, w, 2)
        residual = u_single - u_denoised
        solution_norms.append(np.linalg.norm(u_denoised))
        residual_norms.append(np.linalg.norm(residual))
        
    solution_norms = np.array(solution_norms)
    residual_norms = np.array(residual_norms)
    # Find and extract point of most curvature
    curvature = np.gradient(np.gradient(solution_norms, residual_norms), 
                            residual_norms)
    max_curvature_idx = np.argmax(np.abs(curvature))
    optimal_w = windows[max_curvature_idx]
    
    # Return optimal window size
    return optimal_w

In [ ]:
# Pareto selection of l1 Trend Filter lambda
# Lorenz lambda range: 10^-5 to 10^2
# Burgers lambda range: 10^-5 to 10^1
def pareto_l1_lambda(u_single):
    # Initialization
    lambdas = np.logspace(-5, 2, 50)
    solution_norms = []
    residual_norms = []
    
    # Calculate norms for each window after filter application
    for lambda_reg in lambdas:
        u_denoised = l1_trend_filter(u_single, lambda_reg)
        residual = u_single - u_denoised
        solution_norms.append(np.linalg.norm(u_denoised))
        residual_norms.append(np.linalg.norm(residual))
        
    solution_norms = np.array(solution_norms)
    residual_norms = np.array(residual_norms)
    # Find and extract point of most curvature
    curvature = np.gradient(np.gradient(solution_norms, residual_norms), 
                            residual_norms)
    max_curvature_idx = np.argmax(np.abs(curvature))
    optimal_lambda = lambdas[max_curvature_idx]
    
    # Return optimal lambda
    return optimal_lambda

In [ ]:
# Lorenz Main Test Loop
# Add model printouts for correct equation extractions
# SINDy setup support from [https://pysindy.readthedocs.io/en/latest/
# examples/12_weakform_SINDy_examples/example.html#Test-weak-form-ODE-
# functionality-on-Lorenz-equation]

num_iterations = 500
noise_levels = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70]
true_coeffs_x = {'x': -10, 'y': 10}
true_coeffs_y = {'x': 28, 'y': -1, 'xz': -1}
true_coeffs_z = {'z': -2.666, 'xy': 1}
threshold = 0.0005
# Initialize training data
dt = 0.001
integrator_keywords = {}
feature_names = ['x', 'y', 'z']
t_train = np.arange(0, 8, dt)
x0_train = [-8, 8, 27]
t_train_span = (t_train[0], t_train[-1])
x_train_original = solve_ivp(lorenz, t_train_span, x0_train, t_eval=t_train, 
                             **integrator_keywords).y.T

for p in noise_levels: 
    # Initialize error metrics
    success_count = 0
    success_count_savgol = 0
    success_count_l1 = 0
    success_count_savl1 = 0
    success_count_l1sav = 0
    coeff_errors = []
    coeff_errors_savgol = []
    coeff_errors_l1 = []
    coeff_errors_savl1 = []
    coeff_errors_l1sav = []
    wrong_terms_list = []
    wrong_terms_savgol_list = []
    wrong_terms_l1_list = []
    wrong_terms_savl1_list = []
    wrong_terms_l1sav_list = []
    print(f"Testing noise level: {p}")
    for _ in range(num_iterations):
        # Add Gaussian noise corruption
        x_train = add_noise(x_train_original, p)
        # Initialize candidate library setup
        library_functions = [lambda x: x, lambda x, y: x * y, lambda x: x ** 2]
        library_function_names = [lambda x: x, lambda x, y: x + y, 
                                  lambda x: x + x]
        ode_lib = ps.WeakPDELibrary(library_functions=library_functions, 
                                function_names=library_function_names, 
                                spatiotemporal_grid=t_train, 
                                is_uniform = True,
                                K=100,
                                include_bias=False) # no constant term
        # No filter model
        x_dot_integral = ode_lib.convert_u_dot_integral(x_train)
        model = ps.SINDy(feature_names=feature_names,
                    optimizer=ps.STLSQ(),
                    feature_library=ode_lib)
        model.fit(x_train, x_dot=x_dot_integral,t=dt)
        #print("no filter")
        #model.print()
        coeffs = model.coefficients()
        # Filtering
        num_vars = x_train.shape[1]
        x_train_savgol = np.zeros_like(x_train)
        x_train_l1 = np.zeros_like(x_train)
        x_train_savl1 = np.zeros_like(x_train)
        x_train_l1sav = np.zeros_like(x_train)
        for i in range(num_vars):
            x_pareto_w = lorenz_pareto_savgol_w(x_train[:, i])
            x_train_savgol[:, i] = savgol_denoise_1D(x_train[:, i], 
                                                     x_pareto_w, 2)
            x_pareto_lambda = lorenz_pareto_l1_lambda(x_train[:, i])
            x_train_l1[:, i] = l1_trend_filter(x_train[:, i], x_pareto_lambda)
        for i in range(num_vars):
            savgol_pareto_lambda = lorenz_pareto_l1_lambda(x_train_savgol[:, i])
            x_train_savl1[:, i] = l1_trend_filter(x_train_savgol[:, i], 
                                                  savgol_pareto_lambda)
            l1_pareto_w = lorenz_pareto_savgol_w(x_train_l1[:, i])
            x_train_l1sav[:, i] = savgol_denoise_1D(x_train_l1[:, i], 
                                                    l1_pareto_w, 2)
        # Fitting filtered models
        x_dot_integral_savgol = ode_lib.convert_u_dot_integral(x_train_savgol)    
        x_dot_integral_l1 = ode_lib.convert_u_dot_integral(x_train_l1)    
        x_dot_integral_savl1 = ode_lib.convert_u_dot_integral(x_train_savl1)  
        x_dot_integral_l1sav = ode_lib.convert_u_dot_integral(x_train_l1sav)   
        model_savgol = ps.SINDy(feature_names=feature_names,optimizer=ps.STLSQ(), 
                                feature_library=ode_lib) 
        model_savgol.fit(x_train_savgol, x_dot=x_dot_integral_savgol, t=dt)
        coeffs_savgol = model_savgol.coefficients()
        #print("savgol")
        #model_savgol.print()
        model_l1 = ps.SINDy(feature_names=feature_names,optimizer=ps.STLSQ(), 
                            feature_library=ode_lib)
        model_l1.fit(x_train_l1, x_dot=x_dot_integral_l1, t=dt)
        coeffs_l1 = model_l1.coefficients()
        #print("l1")
        #model_l1.print()
        model_savl1 = ps.SINDy(feature_names=feature_names,optimizer=ps.STLSQ(), 
                               feature_library=ode_lib)
        model_savl1.fit(x_train_savl1, x_dot=x_dot_integral_savl1, t=dt)
        coeffs_savl1 = model_savl1.coefficients()
        #print("savl1")
        #model_savl1.print()
        model_l1sav = ps.SINDy(feature_names=feature_names,optimizer=ps.STLSQ(), 
                               feature_library=ode_lib)
        model_l1sav.fit(x_train_l1sav, x_dot=x_dot_integral_l1sav, t=dt)
        coeffs_l1sav = model_l1sav.coefficients()
        #print("l1sav")
        #model_l1sav.print()
        
        timesteps = np.arange(x_train.shape[0])
        labels = ['x', 'y', 'z']
        all_terms = model.get_feature_names()
        # Desired terms as per given equations
        desired_terms_x = set(['x', 'y'])
        desired_terms_y = set(['x', 'y', 'xz'])
        desired_terms_z = set(['z', 'xy'])
        # Helper function to get active terms for an equation
        def get_active_terms(coeff_row, all_terms):
            return set([term for idx, term in enumerate(all_terms) 
                        if abs(coeff_row[idx]) > threshold])
        # Extract active terms
        active_terms_x = get_active_terms(coeffs[0], all_terms)
        active_terms_y = get_active_terms(coeffs[1], all_terms)
        active_terms_z = get_active_terms(coeffs[2], all_terms)
        active_terms_x_savgol = get_active_terms(coeffs_savgol[0], all_terms)
        active_terms_y_savgol = get_active_terms(coeffs_savgol[1], all_terms)
        active_terms_z_savgol = get_active_terms(coeffs_savgol[2], all_terms)
        active_terms_x_l1 = get_active_terms(coeffs_l1[0], all_terms)
        active_terms_y_l1 = get_active_terms(coeffs_l1[1], all_terms)
        active_terms_z_l1 = get_active_terms(coeffs_l1[2], all_terms)
        active_terms_x_savl1 = get_active_terms(coeffs_savl1[0], all_terms)
        active_terms_y_savl1 = get_active_terms(coeffs_savl1[1], all_terms)
        active_terms_z_savl1 = get_active_terms(coeffs_savl1[2], all_terms)
        active_terms_x_l1sav = get_active_terms(coeffs_l1sav[0], all_terms)
        active_terms_y_l1sav = get_active_terms(coeffs_l1sav[1], all_terms)
        active_terms_z_l1sav = get_active_terms(coeffs_l1sav[2], all_terms)
        # Success Rates
        if (active_terms_x == desired_terms_x and 
            active_terms_y == desired_terms_y and 
            active_terms_z == desired_terms_z):
            success_count += 1
        if (active_terms_x_savgol == desired_terms_x and 
            active_terms_y_savgol == desired_terms_y and 
            active_terms_z_savgol == desired_terms_z):
            success_count_savgol += 1
        if (active_terms_x_l1 == desired_terms_x and 
            active_terms_y_l1 == desired_terms_y and 
            active_terms_z_l1 == desired_terms_z):
            success_count_l1 += 1
        if (active_terms_x_savl1 == desired_terms_x and 
            active_terms_y_savl1 == desired_terms_y and 
            active_terms_z_savl1 == desired_terms_z):
            success_count_savl1 += 1
        if (active_terms_x_l1sav == desired_terms_x and 
            active_terms_y_l1sav == desired_terms_y and 
            active_terms_z_l1sav == desired_terms_z):
            success_count_l1sav += 1
        # Coefficient Errors
        coeff_error = 0
        coeff_error_savgol = 0
        coeff_error_l1 = 0
        coeff_error_savl1 = 0
        coeff_error_l1sav = 0
        for term in desired_terms_x: 
            idx = all_terms.index(term)
            coeff_error += abs(coeffs[0, idx] - true_coeffs_x[term])
            coeff_error_savgol += abs(coeffs_savgol[0, idx] - true_coeffs_x[term])
            coeff_error_l1 += abs(coeffs_l1[0, idx] - true_coeffs_x[term])
            coeff_error_savl1 += abs(coeffs_savl1[0, idx] - true_coeffs_x[term])
            coeff_error_l1sav += abs(coeffs_l1sav[0, idx] - true_coeffs_x[term])
        for term in desired_terms_y: 
            idx = all_terms.index(term)
            coeff_error += abs(coeffs[1, idx] - true_coeffs_y[term])
            coeff_error_savgol += abs(coeffs_savgol[1, idx] - true_coeffs_y[term])
            coeff_error_l1 += abs(coeffs_l1[1, idx] - true_coeffs_y[term])
            coeff_error_savl1 += abs(coeffs_savl1[1, idx] - true_coeffs_y[term])
            coeff_error_l1sav += abs(coeffs_l1sav[1, idx] - true_coeffs_y[term])
        for term in desired_terms_z: 
            idx = all_terms.index(term)
            coeff_error += abs(coeffs[2, idx] - true_coeffs_z[term])
            coeff_error_savgol += abs(coeffs_savgol[2, idx] - true_coeffs_z[term])
            coeff_error_l1 += abs(coeffs_l1[2, idx] - true_coeffs_z[term])
            coeff_error_savl1 += abs(coeffs_savl1[2, idx] - true_coeffs_z[term])
            coeff_error_l1sav += abs(coeffs_l1sav[2, idx] - true_coeffs_z[term])
        coeff_errors.append(coeff_error)
        coeff_errors_savgol.append(coeff_error_savgol)
        coeff_errors_l1.append(coeff_error_l1)
        coeff_errors_savl1.append(coeff_error_savl1)
        coeff_errors_l1sav.append(coeff_error_l1sav)
        # Wrong terms count
        wrong_terms = 0
        wrong_terms += len([term for term in active_terms_x if 
                            term not in desired_terms_x])
        wrong_terms += len([term for term in active_terms_y if
                            term not in desired_terms_y])
        wrong_terms += len([term for term in active_terms_z if
                            term not in desired_terms_z])
        wrong_terms_savgol= 0
        wrong_terms_savgol += len([term for term in active_terms_x_savgol if
                                   term not in desired_terms_x])
        wrong_terms_savgol += len([term for term in active_terms_y_savgol if
                                   term not in desired_terms_y])
        wrong_terms_savgol += len([term for term in active_terms_z_savgol if
                                   term not in desired_terms_z])
        wrong_terms_l1 = 0
        wrong_terms_l1 += len([term for term in active_terms_x_l1 if
                               term not in desired_terms_x])
        wrong_terms_l1 += len([term for term in active_terms_y_l1 if
                               term not in desired_terms_y])
        wrong_terms_l1 += len([term for term in active_terms_z_l1 if
                               term not in desired_terms_z])
        wrong_terms_savl1 = 0
        wrong_terms_savl1 += len([term for term in active_terms_x_savl1 if
                                  term not in desired_terms_x])
        wrong_terms_savl1 += len([term for term in active_terms_y_savl1 if
                                  term not in desired_terms_y])
        wrong_terms_savl1 += len([term for term in active_terms_z_savl1 if
                                  term not in desired_terms_z])
        wrong_terms_l1sav = 0
        wrong_terms_l1sav += len([term for term in active_terms_x_l1sav if
                                  term not in desired_terms_x])
        wrong_terms_l1sav += len([term for term in active_terms_y_l1sav if
                                  term not in desired_terms_y])
        wrong_terms_l1sav += len([term for term in active_terms_z_l1sav if
                                  term not in desired_terms_z])
        wrong_terms_list.append(wrong_terms)
        wrong_terms_savgol_list.append(wrong_terms_savgol)
        wrong_terms_l1_list.append(wrong_terms_l1)
        wrong_terms_savl1_list.append(wrong_terms_savl1)
        wrong_terms_l1sav_list.append(wrong_terms_l1sav)
    # Output metrics
    success_rate = (success_count / num_iterations) * 100
    success_rate_savgol = (success_count_savgol / num_iterations) * 100
    success_rate_l1 = (success_count_l1 / num_iterations) * 100
    success_rate_savl1 = (success_count_savl1 / num_iterations) * 100
    success_rate_l1sav = (success_count_l1sav / num_iterations) * 100
    print(f"Success rate using original data: {success_rate}%")
    print(f"Success rate using savgol data: {success_rate_savgol}%")
    print(f"Success rate using l1 data: {success_rate_l1}%")
    print(f"Success rate using savl1 data: {success_rate_savl1}%")
    print(f"Success rate using l1sav data: {success_rate_l1sav}%")
    print(f"Avg coeff error using original data: {np.mean(coeff_errors)}")
    print(f"Avg coeff error using savgol data: {np.mean(coeff_errors_savgol)}")
    print(f"Avg coeff error using l1 data: {np.mean(coeff_errors_l1)}")
    print(f"Avg coeff error using savl1 data: {np.mean(coeff_errors_savl1)}")
    print(f"Avg coeff error using l1sav data: {np.mean(coeff_errors_l1sav)}")                                                                                  
    print(f"Avg # wrong terms using original data: {np.mean(wrong_terms_list)}")
    print(f"Avg # wrong terms using savgol data: {np.mean(wrong_terms_savgol_list)}")
    print(f"Avg # wrong terms using l1 data: {np.mean(wrong_terms_l1_list)}")
    print(f"Avg # wrong terms using savl1 data: {np.mean(wrong_terms_savl1_list)}")                     
    print(f"Avg # wrong terms using l1sav data: {np.mean(wrong_terms_l1sav_list)}")                                                                                    


In [ ]:
# Viscous Burgers Main Test Loop
# Add model printouts for correct equation extractions
# SINDy setup support from [https://pysindy.readthedocs.io/en/latest/
# examples/12_weakform_SINDy_examples/example.html#Test-weak-form-PDE-
# functionality-on-Burgers'-equation-with-20%-noise]

num_iterations = 500
noise_levels = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]
threshold = 0.0005
true_coeffs = {'x0_11': 0.1, 'x0x0_1': -1.0}

for p in noise_levels: 
    # Initialize error metrics
    success_count = 0
    success_count_savgol = 0
    success_count_l1 = 0
    success_count_savl1 = 0
    success_count_l1sav = 0
    coeff_errors = []
    coeff_errors_savgol = []
    coeff_errors_l1 = []
    coeff_errors_savl1 = []
    coeff_errors_l1sav = []
    wrong_terms_list = []
    wrong_terms_savgol_list = []
    wrong_terms_l1_list = []
    wrong_terms_savl1_list = []
    wrong_terms_l1sav_list = []
    print(f"Testing noise level: {p}")
    for _ in range(num_iterations):
        # Load data file
        data = loadmat("data/burgers.mat")
        time = np.ravel(data["t"])
        x = np.ravel(data["x"])
        u = np.real(data["usol"])
        dt = time[1] - time[0]
        dx = x[1] - x[0]
        # Add Gaussian noise and reshape
        u = add_noise(u, p)
        u = np.reshape(u, (len(x), len(time), 1))
        # Initialize candidate library setup
        library_functions = [lambda x: x, lambda x: x * x]
        library_function_names = [lambda x: x, lambda x: x + x]
        # Define 2D spatiotemporal grid 
        X, T = np.meshgrid(x, time)
        XT = np.asarray([X, T]).T
        pde_lib = ps.WeakPDELibrary(
            library_functions=library_functions,
            function_names=library_function_names,
            derivative_order=2,
            spatiotemporal_grid=XT,
            is_uniform=True,
            K=1000,
        )
        # Filtering
        u_savgol = np.empty_like(u)
        u_l1 = np.empty_like(u)
        u_savl1 = np.empty_like(u)
        u_l1sav = np.empty_like(u)
        u_pareto_lambda = burgers_pareto_l1_lambda(u)
        u_pareto_w = burgers_pareto_savgol_w(u)
        for j in range(u.shape[1]):
            u_savgol[:, j, 0] = savgol_denoise_1D(u[:, j, 0], u_pareto_w, 3)
        for j in range(u.shape[1]):
            u_l1[:, j, 0] = l1_trend_filter(u[:, j, 0], u_pareto_lambda)
        savgol_pareto_lambda = burgers_pareto_l1_lambda(u_savgol)
        l1_pareto_w = burgers_pareto_savgol_w(u_l1)
        for j in range(u.shape[1]):
            u_savl1[:, j, 0] = l1_trend_filter(u_savgol[:, j, 0], savgol_pareto_lambda)
        for j in range(u.shape[1]):
            u_l1sav[:,j,0] = savgol_denoise_1D(u_l1[:, j, 0], l1_pareto_w, 3)
        # Fit all models
        optimizer = ps.SR3(
            threshold=0.1, thresholder="l0", tol=1e-8, 
            normalize_columns=True, max_iter=1000
        )   
        model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
        model_savgol = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
        model_l1 = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
        model_savl1 = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
        model_l1sav = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
        model.fit(u, quiet=True)
        coeffs = model.coefficients()
        #print("no filter")
        #model.print()
        model_savgol.fit(u_savgol, quiet=True)
        coeffs_savgol = model_savgol.coefficients()
        #print("savgol")
        #model_savgol.print()
        model_l1.fit(u_l1, quiet=True)
        coeffs_l1 = model_l1.coefficients()
        #print("l1")
        #model_l1.print()
        model_savl1.fit(u_savl1, quiet=True)
        coeffs_savl1 = model_savl1.coefficients()
        #print("savl1")
        #model_savl1.print()
        model_l1sav.fit(u_l1sav, quiet=True)
        coeffs_l1sav = model_l1sav.coefficients()
        #print("l1sav")
        #model_l1sav.print()
        # Extract active terms
        all_terms = model.get_feature_names()
        non_zero_indices = np.where(np.abs(coeffs) > threshold)[1]
        active_terms = [all_terms[i] for i in non_zero_indices]
        non_zero_indices_savgol = np.where(np.abs(coeffs_savgol) > threshold)[1]
        active_terms_savgol = [all_terms[i] for i in non_zero_indices_savgol]
        non_zero_indices_l1 = np.where(np.abs(coeffs_l1) > threshold)[1]
        active_terms_l1 = [all_terms[i] for i in non_zero_indices_l1]
        non_zero_indices_savl1 = np.where(np.abs(coeffs_savl1) > threshold)[1]
        active_terms_savl1 = [all_terms[i] for i in non_zero_indices_savl1]
        non_zero_indices_l1sav = np.where(np.abs(coeffs_l1sav) > threshold)[1]
        active_terms_l1sav = [all_terms[i] for i in non_zero_indices_l1sav]
        desired_terms = set(['x0_11', 'x0x0_1'])
        # Success Rate
        if set(active_terms) == desired_terms:
            success_count += 1
        if set(active_terms_savgol) == desired_terms:
            success_count_savgol += 1   
        if set(active_terms_l1) == desired_terms:
            success_count_l1 += 1   
        if set(active_terms_savl1) == desired_terms:
            success_count_savl1 += 1   
        if set(active_terms_l1sav) == desired_terms:
            success_count_l1sav += 1   
        # Coefficient Errors
        coeff_error = 0
        coeff_error_savgol = 0
        coeff_error_l1 = 0
        coeff_error_savl1 = 0
        coeff_error_l1sav = 0
        for term in desired_terms:
            idx = all_terms.index(term)
            coeff_error += abs(coeffs[0, idx] - true_coeffs[term])
            coeff_error_savgol += abs(coeffs_savgol[0, idx] - true_coeffs[term])
            coeff_error_l1 += abs(coeffs_l1[0, idx] - true_coeffs[term])
            coeff_error_savl1 += abs(coeffs_savl1[0, idx] - true_coeffs[term])
            coeff_error_l1sav += abs(coeffs_l1sav[0, idx] - true_coeffs[term])
        coeff_errors.append(coeff_error)
        coeff_errors_savgol.append(coeff_error_savgol)
        coeff_errors_l1.append(coeff_error_l1)
        coeff_errors_savl1.append(coeff_error_savl1)
        coeff_errors_l1sav.append(coeff_error_l1sav)
        # Wrong terms count
        wrong_terms = [term for term in active_terms if
                       term not in desired_terms]
        wrong_terms_savgol = [term for term in active_terms_savgol if
                              term not in desired_terms]
        wrong_terms_l1 = [term for term in active_terms_l1 if
                          term not in desired_terms]
        wrong_terms_savl1 = [term for term in active_terms_savl1 if
                             term not in desired_terms]
        wrong_terms_l1sav = [term for term in active_terms_l1sav if
                             term not in desired_terms]
        wrong_terms_list.append(len(wrong_terms))
        wrong_terms_savgol_list.append(len(wrong_terms_savgol))
        wrong_terms_l1_list.append(len(wrong_terms_l1))
        wrong_terms_savl1_list.append(len(wrong_terms_savl1))
        wrong_terms_l1sav_list.append(len(wrong_terms_l1sav))
    # Output metrics
    success_rate = (success_count / num_iterations) * 100
    success_rate_savgol = (success_count_savgol / num_iterations) * 100
    success_rate_l1 = (success_count_l1 / num_iterations) * 100
    success_rate_savl1 = (success_count_savl1 / num_iterations) * 100
    success_rate_l1sav = (success_count_l1sav / num_iterations) * 100
    print(f"Success rate using original data: {success_rate}%")
    print(f"Success rate using savgol data: {success_rate_savgol}%")
    print(f"Success rate using l1 data: {success_rate_l1}%")
    print(f"Success rate using savl1 data: {success_rate_savl1}%")
    print(f"Success rate using l1sav data: {success_rate_l1sav}%")
    print(f"Avg coeff error using original data: {np.mean(coeff_errors)}")
    print(f"Avg coeff error using savgol data: {np.mean(coeff_errors_savgol)}")
    print(f"Avg coeff error using l1 data: {np.mean(coeff_errors_l1)}")
    print(f"Avg coeff error using savl1 data: {np.mean(coeff_errors_savl1)}")
    print(f"Avg coeff error using l1sav data: {np.mean(coeff_errors_l1sav)}")                                                                   
    print(f"Avg # wrong terms using original data: {np.mean(wrong_terms_list)}")
    print(f"Avg # wrong terms using savgol data: {np.mean(wrong_terms_savgol_list)}")
    print(f"Avg # wrong terms using l1 data: {np.mean(wrong_terms_l1_list)}")
    print(f"Avg # wrong terms using savl1 data: {np.mean(wrong_terms_savl1_list)}") 
    print(f"Avg # wrong terms using l1sav data: {np.mean(wrong_terms_l1sav_list)}")